In [101]:
import pandas as pd
import numpy as np
import re
import unicodedata

In [102]:
biographic = pd.read_csv("../DATA/house_biographic.csv")
elections = pd.read_csv("../DATA/house_clean_new.csv")
biographic = biographic[[
    'unaccentedGivenName',
    'unaccentedFamilyName',
    'congressNumber',
    'stateName',
    'parties',
    'electionYear',
    'ageDuringElection'
]]
biographic['unaccentedGivenName'] = biographic['unaccentedGivenName'].str.upper()
biographic['unaccentedFamilyName'] = biographic['unaccentedFamilyName'].str.upper()
biographic

,unaccentedGivenName,unaccentedFamilyName,congressNumber,stateName,parties,electionYear,ageDuringElection
0,NEIL,ABERCROMBIE,99,HI,['Democrat'],1984,46
1,NEIL,ABERCROMBIE,102,HI,['Democrat'],1990,52
2,NEIL,ABERCROMBIE,103,HI,['Democrat'],1992,54
3,NEIL,ABERCROMBIE,104,HI,['Democrat'],1994,56
4,NEIL,ABERCROMBIE,105,HI,['Democrat'],1996,58
...,...,...,...,...,...,...,...
13787,RYAN,ZINKE,114,MT,['Republican'],2014,53
13788,RYAN,ZINKE,115,MT,['Republican'],2016,55
13789,RYAN,ZINKE,118,MT,['Republican'],2022,61
13790,EDWIN,ZSCHAU,98,CA,['Republican'],1982,42


In [103]:
def name_to_abbreviation(name):
    abbrev = {
        "Alabama": "AL",
        "Alaska": "AK",
        "Arizona": "AZ",
        "Arkansas": "AR",
        "California": "CA",
        "Colorado": "CO",
        "Connecticut": "CT",
        "Delaware": "DE",
        "Florida": "FL",
        "Georgia": "GA",
        "Hawaii": "HI",
        "Idaho": "ID",
        "Illinois": "IL",
        "Indiana": "IN",
        "Iowa": "IA",
        "Kansas": "KS",
        "Kentucky": "KY",
        "Louisiana": "LA",
        "Maine": "ME",
        "Maryland": "MD",
        "Massachusetts": "MA",
        "Michigan": "MI",
        "Minnesota": "MN",
        "Mississippi": "MS",
        "Missouri": "MO",
        "Montana": "MT",
        "Nebraska": "NE",
        "Nevada": "NV",
        "New Hampshire": "NH",
        "New Jersey": "NJ",
        "New Mexico": "NM",
        "New York": "NY",
        "North Carolina": "NC",
        "North Dakota": "ND",
        "Ohio": "OH",
        "Oklahoma": "OK",
        "Oregon": "OR",
        "Pennsylvania": "PA",
        "Rhode Island": "RI",
        "South Carolina": "SC",
        "South Dakota": "SD",
        "Tennessee": "TN",
        "Texas": "TX",
        "Utah": "UT",
        "Vermont": "VT",
        "Virginia": "VA",
        "Washington": "WA",
        "West Virginia": "WV",
        "Wisconsin": "WI",
        "Wyoming": "WY",
        "District of Columbia": "DC",
        "American Samoa": "AS",
        "Guam": "GU",
        "Northern Mariana Islands": "MP",
        "Puerto Rico": "PR",
        "United States Minor Outlying Islands": "UM",
        "U.S. Virgin Islands": "VI",
    }
    return abbrev[name.title()]

In [104]:
elections['state'] = elections['state'].apply(name_to_abbreviation)
elections

,year,state,district,democrat_votes,republican_votes,totalvotes,democrat_candidate,republican_candidate,winner,winner_percentage,excess_dem_votes,total_demrep_votes,dem_margin
0,1976,AL,1,58906,98257,157170,BILL DAVENPORT,JACK EDWARDS,Republican,0.625164,-39351,157163,-0.250383
1,1976,AL,2,66288,90069,156362,J CAROLE KEAHEY,"WILLIAM L ""BILL"" DICKINSON",Republican,0.576029,-23781,156357,-0.152094
2,1976,AL,4,141490,34531,176022,TOM BEVILL,LEONARD WILSON,Democrat,0.803820,106959,176021,0.607649
3,1976,AL,6,69384,92113,162518,MEL BAILEY,JOHN H BUCHANAN JR,Republican,0.566786,-22729,161497,-0.140739
4,1976,AK,0,34194,83722,118208,EBEN HOPSON,DON YOUNG,Republican,0.708260,-49528,117916,-0.420028
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8975,2022,WI,3,152977,164743,317922,BRAD PFAFF,DERRICK VAN ORDEN,Republican,0.518187,-11766,317720,-0.037033
8976,2022,WI,4,191955,57660,255012,GWEN MOORE,TIM ROGERS,Democrat,0.752729,134295,249615,0.538009
8977,2022,WI,5,134581,243741,378523,MIKE VAN SOMEREN,SCOTT FITZGERALD,Republican,0.643927,-109160,378322,-0.288537
8978,2022,WI,7,128877,209224,338268,RICHARD DICK AUSMAN,THOMAS P TIFFANY,Republican,0.618515,-80347,338101,-0.237642


In [105]:
def extract_last_name(full_name):
    name_tokens = re.split(r'[ -]', full_name)
    name_tokens = [re.sub(r'[,".]', "", unicodedata.normalize('NFKD', token)) for token in name_tokens]
    if name_tokens[-1] in ("JR", "SR", "I", "II", "III", "IV", "IIII", "V", "JR.", "SR."):
        name_tokens = name_tokens[:-1]
    if len(name_tokens) > 2 and len(name_tokens[-2]) > 1 and '\"' not in name_tokens[-2] and '”' not in name_tokens[
        -2] and name_tokens[-2] != "":
        return [name_tokens[-2], name_tokens[-1]]
    return [name_tokens[-1]]

def extract_first_name(full_name):
    name_tokens = re.split(r'[ -]', full_name)
    name_tokens = [re.sub(r'[,.]', "", unicodedata.normalize('NFKD', token)) for token in name_tokens]
    if name_tokens[-1] in ("JR", "SR", "I", "II", "III", "IV", "IIII", "V", "JR.", "SR."):
        name_tokens = name_tokens[:-1]
    names = []
    if len(name_tokens) < 3:
        names.append(name_tokens[0])
    elif len(name_tokens[1]) > 1 or (len(name_tokens[0]) == 1 and len(name_tokens[1]) == 1):
        names += [name_tokens[0], name_tokens[1]]
    else:
        names.append(name_tokens[0])
    for token in name_tokens:
        if '"' in token and token not in names:
            names.append(token)
    names = [re.sub(r'"', "", name) for name in names]
    return names

In [106]:
elections['dem_last_name'] = elections['democrat_candidate'].apply(extract_last_name)
elections['rep_last_name'] = elections['republican_candidate'].apply(extract_last_name)
elections['dem_first_name'] = elections['democrat_candidate'].apply(extract_first_name)
elections['rep_first_name'] = elections['republican_candidate'].apply(extract_first_name)
elections[['republican_candidate', 'rep_first_name', 'rep_last_name', 'democrat_candidate', 'dem_first_name', 'dem_last_name']]

,republican_candidate,rep_first_name,rep_last_name,democrat_candidate,dem_first_name,dem_last_name
0,JACK EDWARDS,[JACK],[EDWARDS],BILL DAVENPORT,[BILL],[DAVENPORT]
1,"WILLIAM L ""BILL"" DICKINSON","[WILLIAM, BILL]","[BILL, DICKINSON]",J CAROLE KEAHEY,"[J, CAROLE]","[CAROLE, KEAHEY]"
2,LEONARD WILSON,[LEONARD],[WILSON],TOM BEVILL,[TOM],[BEVILL]
3,JOHN H BUCHANAN JR,[JOHN],[BUCHANAN],MEL BAILEY,[MEL],[BAILEY]
4,DON YOUNG,[DON],[YOUNG],EBEN HOPSON,[EBEN],[HOPSON]
...,...,...,...,...,...,...
8975,DERRICK VAN ORDEN,"[DERRICK, VAN]","[VAN, ORDEN]",BRAD PFAFF,[BRAD],[PFAFF]
8976,TIM ROGERS,[TIM],[ROGERS],GWEN MOORE,[GWEN],[MOORE]
8977,SCOTT FITZGERALD,[SCOTT],[FITZGERALD],MIKE VAN SOMEREN,"[MIKE, VAN]","[VAN, SOMEREN]"
8978,THOMAS P TIFFANY,[THOMAS],[TIFFANY],RICHARD DICK AUSMAN,"[RICHARD, DICK]","[DICK, AUSMAN]"


In [107]:
biographic['last_names'] = biographic['unaccentedFamilyName'].apply(extract_last_name)
biographic['first_names'] = biographic['unaccentedGivenName'].apply(extract_first_name)
biographic[biographic['unaccentedGivenName'].str.contains("[- ]")][['unaccentedGivenName', 'first_names', 'unaccentedFamilyName', 'last_names']]

,unaccentedGivenName,first_names,unaccentedFamilyName,last_names
1305,BARBARA-ROSE,[BARBARA],COLLINS,[COLLINS]
1306,BARBARA-ROSE,[BARBARA],COLLINS,[COLLINS]
1307,BARBARA-ROSE,[BARBARA],COLLINS,[COLLINS]
6057,C. A.,[C],RUPPERSBERGER,[RUPPERSBERGER]
6058,C. A.,[C],RUPPERSBERGER,[RUPPERSBERGER]
6059,C. A.,[C],RUPPERSBERGER,[RUPPERSBERGER]
6060,C. A.,[C],RUPPERSBERGER,[RUPPERSBERGER]
6061,C. A.,[C],RUPPERSBERGER,[RUPPERSBERGER]
6062,C. A.,[C],RUPPERSBERGER,[RUPPERSBERGER]
6063,C. A.,[C],RUPPERSBERGER,[RUPPERSBERGER]


In [108]:
biographic[biographic['unaccentedFamilyName'].str.contains("[- ]")]

,unaccentedGivenName,unaccentedFamilyName,congressNumber,stateName,parties,electionYear,ageDuringElection,last_names,first_names
578,LISA,BLUNT ROCHESTER,115,DE,['Democrat'],2016,54,[ROCHESTER],[LISA]
579,LISA,BLUNT ROCHESTER,116,DE,['Democrat'],2018,56,[ROCHESTER],[LISA]
580,LISA,BLUNT ROCHESTER,117,DE,['Democrat'],2020,58,[ROCHESTER],[LISA]
581,LISA,BLUNT ROCHESTER,118,DE,['Democrat'],2022,60,[ROCHESTER],[LISA]
1607,ELIGIO,DE LA GARZA,89,TX,['Democrat'],1964,37,"[LA, GARZA]",[ELIGIO]
...,...,...,...,...,...,...,...,...,...
13352,GUY,VANDER JAGT,98,MI,['Republican'],1982,51,[JAGT],[GUY]
13353,GUY,VANDER JAGT,99,MI,['Republican'],1984,53,[JAGT],[GUY]
13354,GUY,VANDER JAGT,100,MI,['Republican'],1986,55,[JAGT],[GUY]
13355,GUY,VANDER JAGT,101,MI,['Republican'],1988,57,[JAGT],[GUY]


In [109]:
elections

,year,state,district,democrat_votes,republican_votes,totalvotes,democrat_candidate,republican_candidate,winner,winner_percentage,excess_dem_votes,total_demrep_votes,dem_margin,dem_last_name,rep_last_name,dem_first_name,rep_first_name
0,1976,AL,1,58906,98257,157170,BILL DAVENPORT,JACK EDWARDS,Republican,0.625164,-39351,157163,-0.250383,[DAVENPORT],[EDWARDS],[BILL],[JACK]
1,1976,AL,2,66288,90069,156362,J CAROLE KEAHEY,"WILLIAM L ""BILL"" DICKINSON",Republican,0.576029,-23781,156357,-0.152094,"[CAROLE, KEAHEY]","[BILL, DICKINSON]","[J, CAROLE]","[WILLIAM, BILL]"
2,1976,AL,4,141490,34531,176022,TOM BEVILL,LEONARD WILSON,Democrat,0.803820,106959,176021,0.607649,[BEVILL],[WILSON],[TOM],[LEONARD]
3,1976,AL,6,69384,92113,162518,MEL BAILEY,JOHN H BUCHANAN JR,Republican,0.566786,-22729,161497,-0.140739,[BAILEY],[BUCHANAN],[MEL],[JOHN]
4,1976,AK,0,34194,83722,118208,EBEN HOPSON,DON YOUNG,Republican,0.708260,-49528,117916,-0.420028,[HOPSON],[YOUNG],[EBEN],[DON]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8975,2022,WI,3,152977,164743,317922,BRAD PFAFF,DERRICK VAN ORDEN,Republican,0.518187,-11766,317720,-0.037033,[PFAFF],"[VAN, ORDEN]",[BRAD],"[DERRICK, VAN]"
8976,2022,WI,4,191955,57660,255012,GWEN MOORE,TIM ROGERS,Democrat,0.752729,134295,249615,0.538009,[MOORE],[ROGERS],[GWEN],[TIM]
8977,2022,WI,5,134581,243741,378523,MIKE VAN SOMEREN,SCOTT FITZGERALD,Republican,0.643927,-109160,378322,-0.288537,"[VAN, SOMEREN]",[FITZGERALD],"[MIKE, VAN]",[SCOTT]
8978,2022,WI,7,128877,209224,338268,RICHARD DICK AUSMAN,THOMAS P TIFFANY,Republican,0.618515,-80347,338101,-0.237642,"[DICK, AUSMAN]",[TIFFANY],"[RICHARD, DICK]",[THOMAS]


In [110]:
# Create a function to check if any name in a list of names matches with a given last name
def name_matches(name_list, last_name_list):
    for name in name_list:
        for last in last_name_list:
            if name == last:
                return True

# Create an empty list to store the joined rows
joined_rows = []

# Iterate through each row of the elections dataframe
for _, election_row in elections.iterrows():
    # Filter biographic dataframe based on the first three conditions
    matching_bio_rows = biographic[
        (biographic['electionYear'] == election_row['year']) &
        (biographic['stateName'] == election_row['state'])
    ]

    # Further filter based on the fourth condition
    matches = [
        pd.DataFrame([election_row.tolist() + bio_row.tolist()], columns=election_row.index.tolist() + bio_row.index.tolist())
        for _, bio_row in matching_bio_rows.iterrows()
        if (
                (name_matches(bio_row['last_names'], election_row['dem_last_name']) or
                 name_matches(bio_row['last_names'], election_row['rep_last_name'])) and
                (name_matches(bio_row['first_names'], election_row['dem_first_name']) or
                 name_matches(bio_row['first_names'], election_row['rep_first_name']))
        )
    ]
    joined_rows.extend(matches)

# Convert the list of joined rows into a dataframe
joined_df = pd.concat(joined_rows, ignore_index=True)

# Display the first few rows of the joined dataframe
joined_df

,year,state,district,democrat_votes,republican_votes,totalvotes,democrat_candidate,republican_candidate,winner,winner_percentage,...,rep_first_name,unaccentedGivenName,unaccentedFamilyName,congressNumber,stateName,parties,electionYear,ageDuringElection,last_names,first_names
0,1976,AL,2,66288,90069,156362,J CAROLE KEAHEY,"WILLIAM L ""BILL"" DICKINSON",Republican,0.576029,...,"[WILLIAM, BILL]",WILLIAM,DICKINSON,95,AL,['Republican'],1976,51,[DICKINSON],[WILLIAM]
1,1976,AL,4,141490,34531,176022,TOM BEVILL,LEONARD WILSON,Democrat,0.803820,...,[LEONARD],TOM,BEVILL,95,AL,['Democrat'],1976,55,[BEVILL],[TOM]
2,1976,AL,6,69384,92113,162518,MEL BAILEY,JOHN H BUCHANAN JR,Republican,0.566786,...,[JOHN],JOHN,BUCHANAN,95,AL,['Republican'],1976,48,[BUCHANAN],[JOHN]
3,1976,AZ,1,68404,96397,168119,PATRICIA M FULLINWIDER,JOHN J RHODES,Republican,0.573386,...,[JOHN],JOHN,RHODES,95,AZ,['Republican'],1976,60,[RHODES],[JOHN]
4,1976,AZ,2,106054,71765,182128,MORRIS K UDALL,LAIRD GUTTERSEN,Democrat,0.582305,...,[LAIRD],MORRIS,UDALL,95,AZ,['Democrat'],1976,54,[UDALL],[MORRIS]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6891,2022,WA,9,171746,67631,239848,ADAM SMITH,DOUG BASLER,Democrat,0.716062,...,[DOUG],ADAM,SMITH,118,WA,['Democrat'],2022,57,[SMITH],[ADAM]
6892,2022,WV,1,65428,151511,227196,LACY WATSON,CAROL D MILLER,Republican,0.666874,...,[CAROL],CAROL,MILLER,118,WV,['Republican'],2022,72,[MILLER],[CAROL]
6893,2022,WI,1,135825,162610,300867,ANN ROE,BRYAN STEIL,Republican,0.540471,...,[BRYAN],BRYAN,STEIL,118,WI,['Republican'],2022,41,[STEIL],[BRYAN]
6894,2022,WI,2,268740,101890,378537,MARK POCAN,ERIK OLSEN,Democrat,0.709944,...,[ERIK],MARK,POCAN,118,WI,['Democrat'],2022,58,[POCAN],[MARK]


In [111]:
joined_df = joined_df[[
    'year',
    'state',
    'district',
    'democrat_candidate',
    'republican_candidate',
    'total_demrep_votes',
    'winner',
    'dem_margin',
    'unaccentedGivenName',
    'unaccentedFamilyName',
    'ageDuringElection'
]]
joined_df.drop_duplicates(subset=joined_df.columns.difference(['Unnamed: 0']), inplace=True)

# Re-check for duplicates based on state, year, and district in joined_df after removal
remaining_duplicates = joined_df[joined_df.duplicated(subset=['state', 'year', 'district'], keep=False)].sort_values(by=['state', 'year', 'district'])

remaining_duplicates[['democrat_candidate', 'republican_candidate', 'winner', 'unaccentedGivenName', 'unaccentedFamilyName', 'ageDuringElection', 'year', 'state', 'district']]

/var/folders/4w/ryd_r6790xd370fr_0j0l_800000gn/T/ipykernel_3035/4069495298.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  joined_df.drop_duplicates(subset=joined_df.columns.difference(['Unnamed: 0']), inplace=True)


,democrat_candidate,republican_candidate,winner,unaccentedGivenName,unaccentedFamilyName,ageDuringElection,year,state,district
390,ABNER J MIKVA,JOHN E PORTER,Democrat,ABNER,MIKVA,52,1978,IL,10
391,ABNER J MIKVA,JOHN E PORTER,Democrat,JOHN,PORTER,43,1978,IL,10
402,PAUL SIMON,JOHN T ANDERSON,Democrat,PAUL,SIMON,50,1978,IL,24
403,PAUL SIMON,JOHN T ANDERSON,Democrat,JOHN,ANDERSON,56,1978,IL,24
977,DANIEL G DEFOSSE,PHILIP M CRANE,Republican,DANIEL,CRANE,46,1982,IL,12
978,DANIEL G DEFOSSE,PHILIP M CRANE,Republican,PHILIP,CRANE,52,1982,IL,12
1000,CARL D PERKINS,TOM HAMBY,Democrat,CARL,PERKINS,28,1982,KY,7
1001,CARL D PERKINS,TOM HAMBY,Democrat,CARL,PERKINS,70,1982,KY,7
111,RICHARD A TONRY,ROBERT L LIVINGSTON,Democrat,RICHARD,TONRY,41,1976,LA,1
112,RICHARD A TONRY,ROBERT L LIVINGSTON,Democrat,ROBERT,LIVINGSTON,33,1976,LA,1


In [112]:
drop_indices = [391, 403, 977, 1000, 111, 4980, 1660, 5102, 4965]
joined_df = joined_df.drop(drop_indices)

In [113]:
remaining_duplicates = joined_df[joined_df.duplicated(subset=['state', 'year', 'district'], keep=False)].sort_values(by=['state', 'year', 'district'])
remaining_duplicates

,year,state,district,democrat_candidate,republican_candidate,total_demrep_votes,winner,dem_margin,unaccentedGivenName,unaccentedFamilyName,ageDuringElection


In [114]:
joined_df.to_csv("~/Downloads/raw.csv")

In [115]:
joined_df[joined_df['democrat_candidate'].str.contains("ALLARD")]

,year,state,district,democrat_candidate,republican_candidate,total_demrep_votes,winner,dem_margin,unaccentedGivenName,unaccentedFamilyName,ageDuringElection
184,1976,NY,5,ALLARD K LOWENSTEIN,JOHN W WYDLER,181219,Republican,-0.085284,JOHN,WYDLER,52
2222,1992,CA,33,LUCILLE ROYABL-ALLARD,ROBERT GUZMAN,47438,Democrat,0.349551,LUCILLE,ROYBAL-ALLARD,51
2782,1996,CA,33,LUCILLE ROYBAL-ALLARD,JOHN P LEONARD,55625,Democrat,0.707074,LUCILLE,ROYBAL-ALLARD,55
3086,1998,CA,33,LUCILLE ROYBAL-ALLARD,WAYNE MILLER,49674,Democrat,0.743769,LUCILLE,ROYBAL-ALLARD,57
3347,2000,CA,33,LUCILLE ROYBAL-ALLARD,WAYNE MILLER,68770,Democrat,0.759779,LUCILLE,ROYBAL-ALLARD,59
3635,2002,CA,34,LUCILLE ROYBAL-ALLARD,WAYNE MILLER,65824,Democrat,0.480737,LUCILLE,ROYBAL-ALLARD,61
3905,2004,CA,34,LUCILLE ROYBAL-ALLARD,WAYNE MILLER,110457,Democrat,0.489847,LUCILLE,ROYBAL-ALLARD,63
4182,2006,CA,34,LUCILLE ROYBAL-ALLARD,WAYNE MILLER,74818,Democrat,0.535967,LUCILLE,ROYBAL-ALLARD,65
4483,2008,CA,34,LUCILLE ROYBAL-ALLARD,CHRISTOPHER BALDING,127769,Democrat,0.541892,LUCILLE,ROYBAL-ALLARD,67
4803,2010,CA,34,LUCILLE ROYBAL-ALLARD,WAYNE MILLER,89839,Democrat,0.544585,LUCILLE,ROYBAL-ALLARD,69


In [116]:
joined_df['democrat_candidate'] = joined_df['democrat_candidate'].str.replace("-", " ")
joined_df['republican_candidate'] = joined_df['republican_candidate'].str.replace("-", " ")
joined_df['unaccentedFamilyName'] = joined_df['unaccentedFamilyName'].str.replace("-", " ")
joined_df['unaccentedFamilyName'] = joined_df['unaccentedFamilyName'].str.replace(".", "")
joined_df['unaccentedFamilyName'] = joined_df['unaccentedFamilyName'].str.replace(",", "")
joined_df.loc[2222, 'democrat_candidate'] = "ROYBAL ALLARD"
joined_df[joined_df.apply(lambda row: row['unaccentedFamilyName'] not in row['democrat_candidate'] and row['unaccentedFamilyName'] not in row['republican_candidate'], axis=1)]

,year,state,district,democrat_candidate,republican_candidate,total_demrep_votes,winner,dem_margin,unaccentedGivenName,unaccentedFamilyName,ageDuringElection


In [117]:
prior_winners = joined_df.copy()
prior_winners['year'] = prior_winners['year'] + 2  # Shift the year by two years

# Extract only the necessary columns
prior_winners = prior_winners[['year', 'state', 'district', 'winner', 'unaccentedGivenName', 'unaccentedFamilyName', 'ageDuringElection']]

# Rename the columns for clarity when merging later
prior_winners.columns = ['year', 'state', 'district', 'prior_winner_party', 'prior_winner_given_name', 'prior_winner_family_name', 'previous_winner_age']

prior_winners

,year,state,district,prior_winner_party,prior_winner_given_name,prior_winner_family_name,previous_winner_age
0,1978,AL,2,Republican,WILLIAM,DICKINSON,51
1,1978,AL,4,Democrat,TOM,BEVILL,55
2,1978,AL,6,Republican,JOHN,BUCHANAN,48
3,1978,AZ,1,Republican,JOHN,RHODES,60
4,1978,AZ,2,Democrat,MORRIS,UDALL,54
...,...,...,...,...,...,...,...
6891,2024,WA,9,Democrat,ADAM,SMITH,57
6892,2024,WV,1,Republican,CAROL,MILLER,72
6893,2024,WI,1,Republican,BRYAN,STEIL,41
6894,2024,WI,2,Democrat,MARK,POCAN,58


In [118]:
with_prior_winners = pd.merge(joined_df, prior_winners, on=['year', 'state', 'district'], how='left')
with_prior_winners

,year,state,district,democrat_candidate,republican_candidate,total_demrep_votes,winner,dem_margin,unaccentedGivenName,unaccentedFamilyName,ageDuringElection,prior_winner_party,prior_winner_given_name,prior_winner_family_name,previous_winner_age
0,1976,AL,2,J CAROLE KEAHEY,"WILLIAM L ""BILL"" DICKINSON",156357,Republican,-0.152094,WILLIAM,DICKINSON,51,NaN,NaN,NaN,NaN
1,1976,AL,4,TOM BEVILL,LEONARD WILSON,176021,Democrat,0.607649,TOM,BEVILL,55,NaN,NaN,NaN,NaN
2,1976,AL,6,MEL BAILEY,JOHN H BUCHANAN JR,161497,Republican,-0.140739,JOHN,BUCHANAN,48,NaN,NaN,NaN,NaN
3,1976,AZ,1,PATRICIA M FULLINWIDER,JOHN J RHODES,164801,Republican,-0.169859,JOHN,RHODES,60,NaN,NaN,NaN,NaN
4,1976,AZ,2,MORRIS K UDALL,LAIRD GUTTERSEN,177819,Democrat,0.192831,MORRIS,UDALL,54,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6826,2022,WA,9,ADAM SMITH,DOUG BASLER,239377,Democrat,0.434942,ADAM,SMITH,57,Democrat,ADAM,SMITH,55.0
6827,2022,WV,1,LACY WATSON,CAROL D MILLER,216939,Republican,-0.396807,CAROL,MILLER,72,Republican,DAVID,MCKINLEY,73.0
6828,2022,WI,1,ANN ROE,BRYAN STEIL,298435,Republican,-0.089752,BRYAN,STEIL,41,Republican,BRYAN,STEIL,39.0
6829,2022,WI,2,MARK POCAN,ERIK OLSEN,370630,Democrat,0.450179,MARK,POCAN,58,Democrat,MARK,POCAN,56.0


In [119]:
def determine_incumbent(row):
    if pd.isnull(row['prior_winner_family_name']):
        return "None"
    if row['prior_winner_family_name'] in row['democrat_candidate']:
        return "Democrat"
    if row['prior_winner_family_name'] in row['republican_candidate']:
        return "Republican"
    return "None"

# Apply the determine_incumbent function to each row
with_prior_winners['incumbent'] = with_prior_winners.apply(determine_incumbent, axis=1)
with_prior_winners = with_prior_winners.sort_values(by=['state', 'district', 'year'])
with_prior_winners

,year,state,district,democrat_candidate,republican_candidate,total_demrep_votes,winner,dem_margin,unaccentedGivenName,unaccentedFamilyName,ageDuringElection,prior_winner_party,prior_winner_given_name,prior_winner_family_name,previous_winner_age,incumbent
621,1980,AK,0,"KEVIN ""PAT"" PARNELL",DONALD E YOUNG,154011,Republican,-0.481569,DONALD,YOUNG,47,NaN,NaN,NaN,NaN,None
1163,1984,AK,0,PEGGE BEGICH,DONALD E YOUNG,199634,Republican,-0.137902,DONALD,YOUNG,51,NaN,NaN,NaN,NaN,None
5391,2014,AL,1,BURTON R LEFLORE,BRADLEY BYRNE,152036,Republican,-0.364914,BRADLEY,BYRNE,59,NaN,NaN,NaN,NaN,None
5999,2018,AL,1,ROBERT KENNEDY JR,BRADLEY BYRNE,242454,Republican,-0.263976,BRADLEY,BYRNE,63,NaN,NaN,NaN,NaN,None
0,1976,AL,2,J CAROLE KEAHEY,"WILLIAM L ""BILL"" DICKINSON",156357,Republican,-0.152094,WILLIAM,DICKINSON,51,NaN,NaN,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5390,2012,WY,0,CHRIS HENRICHSEN,CYNTHIA M LUMMIS,224025,Republican,-0.486013,CYNTHIA,LUMMIS,58,Republican,CYNTHIA,LUMMIS,56.0,Republican
5688,2014,WY,0,RICHARD GRAYSON,CYNTHIA M LUMMIS,150841,Republican,-0.498770,CYNTHIA,LUMMIS,60,Republican,CYNTHIA,LUMMIS,58.0,Republican
5998,2016,WY,0,RYAN GREENE,LIZ CHENEY,231642,Republican,-0.348426,LIZ,CHENEY,50,Republican,CYNTHIA,LUMMIS,60.0,None
6314,2018,WY,0,GREG HUNTER,LIZ CHENEY,187866,Republican,-0.362279,LIZ,CHENEY,52,Republican,LIZ,CHENEY,50.0,Republican


In [120]:
with_prior_winners['incumbency_count'] = 0

last_winner = {}
last_index = {}

# Iterate over rows to calculate incumbency_count
for index, row in with_prior_winners.iterrows():
    key = (row['state'], row['district'])

    # If there's an incumbent
    if row['incumbent'] != 'None':
        # If the incumbent was the winner in the last election for that district
        if key in last_winner and last_winner[key] == (row['unaccentedGivenName'], row['unaccentedFamilyName']):
            with_prior_winners.at[index, 'incumbency_count'] = with_prior_winners.at[last_index[key], 'incumbency_count'] + 1

    # Update the last winner and index for the district
    if row['winner'] == 'Democrat':
        last_winner[key] = (row['unaccentedGivenName'], row['unaccentedFamilyName'])
    elif row['winner'] == 'Republican':
        winner_name = row['republican_candidate'].split()
        last_winner[key] = (winner_name[0], winner_name[-1])

    last_index[key] = index

with_prior_winners

,year,state,district,democrat_candidate,republican_candidate,total_demrep_votes,winner,dem_margin,unaccentedGivenName,unaccentedFamilyName,ageDuringElection,prior_winner_party,prior_winner_given_name,prior_winner_family_name,previous_winner_age,incumbent,incumbency_count
621,1980,AK,0,"KEVIN ""PAT"" PARNELL",DONALD E YOUNG,154011,Republican,-0.481569,DONALD,YOUNG,47,NaN,NaN,NaN,NaN,None,0
1163,1984,AK,0,PEGGE BEGICH,DONALD E YOUNG,199634,Republican,-0.137902,DONALD,YOUNG,51,NaN,NaN,NaN,NaN,None,0
5391,2014,AL,1,BURTON R LEFLORE,BRADLEY BYRNE,152036,Republican,-0.364914,BRADLEY,BYRNE,59,NaN,NaN,NaN,NaN,None,0
5999,2018,AL,1,ROBERT KENNEDY JR,BRADLEY BYRNE,242454,Republican,-0.263976,BRADLEY,BYRNE,63,NaN,NaN,NaN,NaN,None,0
0,1976,AL,2,J CAROLE KEAHEY,"WILLIAM L ""BILL"" DICKINSON",156357,Republican,-0.152094,WILLIAM,DICKINSON,51,NaN,NaN,NaN,NaN,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5390,2012,WY,0,CHRIS HENRICHSEN,CYNTHIA M LUMMIS,224025,Republican,-0.486013,CYNTHIA,LUMMIS,58,Republican,CYNTHIA,LUMMIS,56.0,Republican,2
5688,2014,WY,0,RICHARD GRAYSON,CYNTHIA M LUMMIS,150841,Republican,-0.498770,CYNTHIA,LUMMIS,60,Republican,CYNTHIA,LUMMIS,58.0,Republican,3
5998,2016,WY,0,RYAN GREENE,LIZ CHENEY,231642,Republican,-0.348426,LIZ,CHENEY,50,Republican,CYNTHIA,LUMMIS,60.0,None,0
6314,2018,WY,0,GREG HUNTER,LIZ CHENEY,187866,Republican,-0.362279,LIZ,CHENEY,52,Republican,LIZ,CHENEY,50.0,Republican,1


In [121]:
with_prior_winners['avg_dem_margin_4'] = 0.0
with_prior_winners['avg_total_votes_4'] = 0.0
with_prior_winners['party_flips_4'] = 0

for index, row in with_prior_winners.iterrows():
    prev_rows = with_prior_winners[
        (with_prior_winners['state'] == row['state']) &
        (with_prior_winners['district'] == row['district']) &
        (with_prior_winners['year'] < row['year'])
    ].tail(4)

    if not prev_rows.empty:
        with_prior_winners.at[index, 'avg_dem_margin_4'] = prev_rows['dem_margin'].mean()
        with_prior_winners.at[index, 'avg_total_votes_4'] = prev_rows['total_demrep_votes'].mean()
        with_prior_winners.at[index, 'party_flips_4'] = (prev_rows['winner'] != prev_rows['winner'].shift()).sum() - 1

with_prior_winners

,year,state,district,democrat_candidate,republican_candidate,total_demrep_votes,winner,dem_margin,unaccentedGivenName,unaccentedFamilyName,ageDuringElection,prior_winner_party,prior_winner_given_name,prior_winner_family_name,previous_winner_age,incumbent,incumbency_count,avg_dem_margin_4,avg_total_votes_4,party_flips_4
621,1980,AK,0,"KEVIN ""PAT"" PARNELL",DONALD E YOUNG,154011,Republican,-0.481569,DONALD,YOUNG,47,NaN,NaN,NaN,NaN,None,0,0.000000,0.00,0
1163,1984,AK,0,PEGGE BEGICH,DONALD E YOUNG,199634,Republican,-0.137902,DONALD,YOUNG,51,NaN,NaN,NaN,NaN,None,0,-0.481569,154011.00,0
5391,2014,AL,1,BURTON R LEFLORE,BRADLEY BYRNE,152036,Republican,-0.364914,BRADLEY,BYRNE,59,NaN,NaN,NaN,NaN,None,0,0.000000,0.00,0
5999,2018,AL,1,ROBERT KENNEDY JR,BRADLEY BYRNE,242454,Republican,-0.263976,BRADLEY,BYRNE,63,NaN,NaN,NaN,NaN,None,0,-0.364914,152036.00,0
0,1976,AL,2,J CAROLE KEAHEY,"WILLIAM L ""BILL"" DICKINSON",156357,Republican,-0.152094,WILLIAM,DICKINSON,51,NaN,NaN,NaN,NaN,None,0,0.000000,0.00,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5390,2012,WY,0,CHRIS HENRICHSEN,CYNTHIA M LUMMIS,224025,Republican,-0.486013,CYNTHIA,LUMMIS,58,Republican,CYNTHIA,LUMMIS,56.0,Republican,2,-0.182703,208296.75,0
5688,2014,WY,0,RICHARD GRAYSON,CYNTHIA M LUMMIS,150841,Republican,-0.498770,CYNTHIA,LUMMIS,60,Republican,CYNTHIA,LUMMIS,58.0,Republican,3,-0.269611,206279.00,0
5998,2016,WY,0,RYAN GREENE,LIZ CHENEY,231642,Republican,-0.348426,LIZ,CHENEY,50,Republican,CYNTHIA,LUMMIS,60.0,None,0,-0.392941,197574.25,0
6314,2018,WY,0,GREG HUNTER,LIZ CHENEY,187866,Republican,-0.362279,LIZ,CHENEY,52,Republican,LIZ,CHENEY,50.0,Republican,1,-0.454327,195984.25,0


In [126]:
training_data = with_prior_winners.loc[(with_prior_winners['year'] > 1976) & (with_prior_winners['avg_total_votes_4'] > 1) & (with_prior_winners['previous_winner_age'] > 1)]
training_data = training_data[['year',
                               'state',
                               'previous_winner_age',
                               'incumbent',
                               'incumbency_count',
                               'avg_dem_margin_4',
                               'avg_total_votes_4',
                               'party_flips_4',
                               'dem_margin'
                               ]]
training_data

,year,state,previous_winner_age,incumbent,incumbency_count,avg_dem_margin_4,avg_total_votes_4,party_flips_4,dem_margin
318,1978,AL,51.0,Republican,1,-0.152094,156357.00,0,-0.080017
4707,2010,AL,56.0,Democrat,0,-0.062914,180365.00,1,-0.021875
5054,2012,AL,34.0,Republican,1,-0.030359,195903.25,2,-0.273189
5392,2014,AL,36.0,Republican,2,-0.078652,240007.75,2,-0.348109
5689,2016,AL,38.0,Republican,3,-0.159234,239233.50,1,-0.092305
...,...,...,...,...,...,...,...,...,...
5390,2012,WY,56.0,Republican,2,-0.182703,208296.75,0,-0.486013
5688,2014,WY,58.0,Republican,3,-0.269611,206279.00,0,-0.498770
5998,2016,WY,60.0,None,0,-0.392941,197574.25,0,-0.348426
6314,2018,WY,50.0,Republican,1,-0.454327,195984.25,0,-0.362279


In [128]:
training_data.to_csv("../DATA/training.csv")